## Installation Requirements and Module Configuration

If you would like to run in your own environment, the following dependencies are required

In [1]:
##Add dependencies here
# !pip install azure-storage-blob

In [28]:
#to reflect changes made in modules - for assembling the notebooks before the workshop - remove when in use
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
##imports
import os, sys
sys.path.append(os.path.join(sys.path[0], '../'))

from columbus_wildlife_workshop import Flickr
fr = Flickr() 

from image_data import Save
s = Save()

## Query Terms

In [115]:
term = 'Columbus AND Ohio AND wildlife AND NOT zoo AND NOT domestic'

## Flickr API

Connect to Flickr photos search api 
API will retrieve results for term selected and download them to a zip file specified

In [116]:
json_res = fr.search(term, saveTo=True)

Hello!!
<Response [200]>
12 Found with Columbus+AND+Ohio+AND+wildlife+AND+NOT+zoo+AND+NOT+domestic
Done Retrieving Flickr Results


In [126]:
len(json_res[0])

97

## Download Images to A Folder
In the same directory this notebook is in, create a folder "images".
This folder will contain all jpg images from the Flickr API query

In [8]:
## For Megadetector images
# file_path = "images/"
# list_images = s.download_images(file_path, json_res)

Save images to a folder specied. As we save images, we build a list of dictionaries representing the images.

For each image, we construct a dictionary with:
   - url: img url from flickr
   - name: name of image
   - data: bytearray representation of img 

In [149]:
# Test out with a small portion of images
subset = json_res[0][0:6]
print(subset[0]['url_l'])

https://live.staticflickr.com/65535/50567923271_926d37d247_b.jpg


In [150]:
# For Species Classfier Images 
file_path = 'subset_classifier_images/'#"classifier_images/"
list_images = s.download_images(file_path, subset)

done downloading images!


In [152]:
len(list_images)

6

## Species Classfication

Connect to Microsoft Azure SpeciesClassification API to generate predictions on species in frame using the photos collected from the Flickr API

In [ ]:
## FIXME: will we have a general API key for all users to use
## or will every individual have to make their own request

In [192]:
# get list of image dicts
# FIXME: try and save this img_data to a json file so we dont
# have to reload each time
image_data = list_images
print(type(image_data))

<class 'list'>


In [177]:
img = img_data[0]['url']

In [183]:
from columbus_wildlife_workshop import SpeciesClassifier

In [193]:
cs = SpeciesClassifier()
for image in image_data:
    cs.classify_and_display_results(image)#image_data[i])

Running API...
...done


result:  ({'predictions': [{'class': 'Aves', 'class_common': 'Birds', 'confidence': 48.289334774017334, 'family': 'Alcedinidae', 'family_common': 'Kingfishers', 'genus': 'Megaceryle', 'genus_common': '', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Coraciiformes', 'order_common': 'Kingfishers and Allies', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Megaceryle alcyon', 'species_common': 'Belted Kingfisher', 'subphylum': 'Vertebrata', 'subphylum_common': 'Vertebrates'}, {'class': 'Aves', 'class_common': 'Birds', 'confidence': 23.83272796869278, 'family': 'Alcedinidae', 'family_common': 'Kingfishers', 'genus': 'Chloroceryle', 'genus_common': 'American Green Kingfishers', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Coraciiformes', 'order_common': 'Kingfishers and Allies', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Chloroceryle amazona', 'species_common': 'Amazon Kingfisher', 'subphylum': 'Vertebrata', 'subphylum_co

Running API...
...done


result:  ({'predictions': [{'class': 'Mammalia', 'class_common': 'Mammals', 'confidence': 11.987053602933884, 'family': 'Cervidae', 'family_common': 'Deer', 'genus': 'Odocoileus', 'genus_common': 'White tailed and Mule Deer', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Artiodactyla', 'order_common': 'Even toed Ungulates', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Odocoileus virginianus', 'species_common': 'White tailed Deer', 'suborder': 'Ruminantia', 'suborder_common': 'Ruminants', 'subphylum': 'Vertebrata', 'subphylum_common': 'Vertebrates'}, {'class': 'Mammalia', 'class_common': 'Mammals', 'confidence': 10.599566996097565, 'family': 'Cervidae', 'family_common': 'Deer', 'genus': 'Odocoileus', 'genus_common': 'White tailed and Mule Deer', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Artiodactyla', 'order_common': 'Even toed Ungulates', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Odocoileus virginianus clavium

Running API...
...done


...

result:  ({'predictions': [{'class': 'Mammalia', 'class_common': 'Mammals', 'confidence': 30.397072434425354, 'family': 'Cervidae', 'family_common': 'Deer', 'genus': 'Odocoileus', 'genus_common': 'White tailed and Mule Deer', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Artiodactyla', 'order_common': 'Even toed Ungulates', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Odocoileus hemionus columbianus', 'species_common': 'Columbian Black tailed Deer', 'suborder': 'Ruminantia', 'suborder_common': 'Ruminants', 'subphylum': 'Vertebrata', 'subphylum_common': 'Vertebrates'}, {'class': 'Mammalia', 'class_common': 'Mammals', 'confidence': 11.539672315120697, 'family': 'Cervidae', 'family_common': 'Deer', 'genus': 'Odocoileus', 'genus_common': 'White tailed and Mule Deer', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Artiodactyla', 'order_common': 'Even toed Ungulates', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Odocoileus 

Running API...
...done


result:  ({'predictions': [{'class': 'Aves', 'class_common': 'Birds', 'confidence': 19.664740562438965, 'family': 'Ardeidae', 'family_common': 'Herons, Egrets, and Bitterns', 'genus': 'Egretta', 'genus_common': '', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Pelecaniformes', 'order_common': 'Pelicans, Herons, Ibises, and Allies', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Egretta garzetta', 'species_common': 'Little Egret', 'subfamily': 'Ardeinae', 'subfamily_common': '', 'subphylum': 'Vertebrata', 'subphylum_common': 'Vertebrates'}, {'class': 'Aves', 'class_common': 'Birds', 'confidence': 15.311738848686218, 'family': 'Ardeidae', 'family_common': 'Herons, Egrets, and Bitterns', 'genus': 'Ardea', 'genus_common': 'Great Herons', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Pelecaniformes', 'order_common': 'Pelicans, Herons, Ibises, and Allies', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Ardea herodias', 'specie

Running API...
...done


result:  ({'predictions': [{'class': 'Aves', 'class_common': 'Birds', 'confidence': 95.41850090026855, 'family': 'Cardinalidae', 'family_common': 'Cardinals and Allies', 'genus': 'Cardinalis', 'genus_common': 'Typical Cardinals', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Passeriformes', 'order_common': 'Perching Birds', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Cardinalis cardinalis', 'species_common': 'Northern Cardinal', 'subphylum': 'Vertebrata', 'subphylum_common': 'Vertebrates'}, {'class': 'Aves', 'class_common': 'Birds', 'confidence': 3.2150261104106903, 'family': 'Cardinalidae', 'family_common': 'Cardinals and Allies', 'genus': 'Cardinalis', 'genus_common': 'Typical Cardinals', 'kingdom': 'Animalia', 'kingdom_common': 'Animals', 'order': 'Passeriformes', 'order_common': 'Perching Birds', 'phylum': 'Chordata', 'phylum_common': 'Chordates', 'species': 'Cardinalis sinuatus', 'species_common': 'Pyrrhuloxia', 'subphylum': 'Vertebrata', 'subph